# Hands-on

## Setup

In [1]:
# ffmpeg - Working with videos, audios, and streaming media files
!sudo apt install -y ffmpeg
# mediapy - dev dependency; displays videos/images in the notebook
!pip install -q mediapy
!pip uninstall -q -y opencv-python-headless
!pip install -q "opencv-python-headless<4.3"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 MB 12.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.3.1 requires opencv-python-headless>=4.1.1, but you have opencv-python-headless 3.4.18.65 which is incompatible.
qudida 0.0.4 requires opencv-python-headless>=4.0.1, but you have opencv-python-headless 3.4.18.65 which is incompatible.


In [2]:
# Import libraries
import pathlib

import matplotlib as mpl
import matplotlib.pyplot as plt
import mediapy as media
import numpy as np
import PIL

import tensorflow as tf
import tensorflow_hub as hub
import tqdm

mpl.rcParams.update({
    'font.size': 10,
})

In [3]:
# Getting the kinetics 600 label
# Printing the first few labels

labels_path = tf.keras.utils.get_file(
    fname="labels.txt",
    origin="https://raw.githubusercontent.com/tensorflow/models/f8af2291cced43fc9f1d9b41ddbf772ae7b0d7d2/official/projects/movinet/files/kinetics_600_labels.txt"
)
labels_path = pathlib.Path(labels_path)

lines = labels_path.read_text().splitlines()
KINETICS_600_LABELS = np.array([line.strip() for line in lines])
KINETICS_600_LABELS[:20]

9209/9209 [==============================] - 0s 0us/step


array(['abseiling', 'acting in play', 'adjusting glasses', 'air drumming',
       'alligator wrestling', 'answering questions', 'applauding',
       'applying cream', 'archaeological excavation', 'archery',
       'arguing', 'arm wrestling', 'arranging flowers',
       'assembling bicycle', 'assembling computer',
       'attending conference', 'auctioning', 'backflip (human)',
       'baking cookies', 'bandaging'], dtype='<U49')

In [4]:
cartwheel_url = 'https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExZHoxc3JlZjA3bTg0aTZzNzBmdnk0enNpdm1haTVxajZmMDRpNGN4cyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/IhsEJ88BJ3MQl9UAoZ/giphy.gif'
cartwheel_path = tf.keras.utils.get_file(
    fname='cartwheel.gif',
    origin=cartwheel_url,
    cache_dir='.',
    cache_subdir='.',
)

2280185/2280185 [==============================] - 0s 0us/step


In [5]:
# @title Loading a gif
def load_gif(file_path, image_size=(224, 224)):
    # Load a gif file, convert it to a TF tensor
    raw = tf.io.read_file(file_path)
    video = tf.io.decode_gif(raw)
    # Resize the video
    video = tf.image.resize(video, image_size)
    # change dtype to a float32
    # Hub models always want images ormalized to [0, 1]
    video = tf.cast(video, tf.float32) / 255
    return video

In [6]:
cartwheel = load_gif(cartwheel_path)
cartwheel.shape

TensorShape([76, 224, 224, 3])

## How to use the model

### Base model

In [7]:
%%time
id = 'a2'
mode = 'base'
version = '3'
hub_url = f'https://tfhub.dev/tensorflow/movinet/{id}/{mode}/kinetics-600/classification/{version}'
model = hub.load(hub_url)

CPU times: user 24.8 s, sys: 1.23 s, total: 26 s
Wall time: 32.1 s


In [ ]:
sig = model.signatures['serving_default']
print(sig.pretty_printed_signature())

In [ ]:
# Adding the outer batch dimension to the video
# warmup
sig(image = cartwheel[tf.newaxis, :1])

In [12]:
%%time
logits = sig(image = cartwheel[tf.newaxis, ...])
logits = logits['classifier_head'][0]

print(logits.shape)
print()

(600,)

CPU times: user 3min 49s, sys: 2.57 s, total: 3min 52s
Wall time: 2min 36s


In [15]:
#@title Get top k probabilities from predictions
def get_top_k(probs, k=5, label_map=KINETICS_600_LABELS):
    # Sort predictions to find top_k
    top_predictions = tf.argsort(probs, axis=-1, direction='DESCENDING')[:k]
    # Collect labels of top_k predictions
    top_labels = tf.gather(label_map, top_predictions, axis=-1)
    # Decode labels
    top_labels = [label.decode('utf-8') for label in top_labels.numpy()]
    # top_k probabilities of the predictions
    top_probs = tf.gather(probs, top_predictions, axis=-1).numpy()
    return tuple(zip(top_labels, top_probs))


In [29]:
probs = tf.nn.softmax(logits, axis=-1)
for label, p in get_top_k(probs):
    print(f'{label:^20s}: {p:.3f}')

    cartwheeling    : 0.841
gymnastics tumbling : 0.046
 standing on hands  : 0.021
   somersaulting    : 0.014
      capoeira      : 0.012


In [35]:
media.show_video(cartwheel.numpy(), fps=5)

This browser does not support the video tag.


In [47]:
temp = cartwheel[tf.newaxis, ...]
len(cartwheel)

76